In [ ]:
import pandas as pd
from pathlib import Path
from drevalpy.datasets.curvecurator import preprocess, _exec_curvecurator, postprocess
import drevalpy
drevalpy.__version__

## Load Data

In [ ]:
RAW_FILE_PATH = Path("beataml_wv1to4_raw_inhibitor_v4_dbgap.txt")
raw_df = pd.read_csv(RAW_FILE_PATH, sep="\t")
raw_df = raw_df.drop(columns=["dbgap_dnaseq_sample", "dbgap_rnaseq_sample", "run_index", "plate_num", "time_of_read", "passed_qc"])
raw_df = raw_df.drop_duplicates()
raw_df.head()

In [ ]:
raw_df["rounddose"] = raw_df["well_concentration"].round(10)
raw_df["mindose"] = raw_df.groupby(["dbgap_subject_id", "inhibitor"], as_index=False)["rounddose"].transform(min)
raw_df["maxdose"] = raw_df.groupby(["dbgap_subject_id", "inhibitor"], as_index=False)["rounddose"].transform(max)
raw_df["repl"] = raw_df.groupby(["dbgap_subject_id", "inhibitor", "rounddose"], as_index=False)["rounddose"].cumcount()
raw_df.head()

In [ ]:
drug_dosages = raw_df[["dbgap_subject_id", "inhibitor", "mindose", "maxdose"]].drop_duplicates()
drug_dosages.to_csv("drug_dosages_BeatAML2.csv", index=False)

### Drug name to PubChem mapping

In [ ]:
import pubchempy as pcp
results = {}
chembls = {}
manual_review = {}
inchis = {}
i = 0
for drug in raw_df['inhibitor'].unique():
    if i % 10 == 0:
        print(i, '/', len(raw_df['inhibitor'].unique()))
    compounds = pcp.get_compounds(identifier=drug, namespace="name")
    if len(compounds) == 0:
        manual_review[drug] = [compounds]
    else:
        results[drug] = compounds[0].cid
        inchis = compounds[0].inchi
        chembl = None
        for syn in compounds[0].synonyms or []:
            if syn.startswith("CHEMBL"):
                chembl = syn
                break
        chembls[drug] = chembl
    i += 1

In [ ]:
print(f'{len(results)} matches, {len(manual_review)} not found')
manual_review |= {brd: cid for brd, cid in results.items() if cid is None}
manual_review

In [ ]:
manual_review['Pazopanib (GW786034)'] = '10113978' # is Pazopanib
manual_review['Roscovitine (CYC-202)'] = '160355' # is Roscovitine
manual_review['Afatinib (BIBW-2992)'] = '10184653' # is Afatinib
manual_review['Axitinib (AG-013736)'] = '6450551' # is Axitinib
manual_review['Foretinib (XL880)'] = '42642645' # is Foretinib
manual_review['Lestaurtinib (CEP-701)'] = '126565' # is Lestaurtinib
manual_review['Masitinib (AB-1010)'] = '10074640' # is Masitinib
manual_review['Vargetef'] = '135423438' # typo (Vargatef) = Nintedanib
manual_review['Vemurafenib (PLX-4032)'] = '42611257' # is Vemurafenib
manual_review['Volasertib (BI-6727)'] = '10461508' # is Volasertib
manual_review['AT-101'] = '3503' # Gossypol; AT 101 is synonym
manual_review['Baiclein'] = '5281605' # typo, is Baicalein
manual_review['Dovitinib (CHIR-258)'] = '135398510' # was empty

In [ ]:
def sanitize(val):
    if isinstance(val, int):
        return str(val)
    if isinstance(val, list):
        return ','.join([str(i) for i in val])
    return val
cids = pd.Series(results | manual_review, name="CID").apply(sanitize).astype(str)
cids

In [ ]:
chembls = pd.Series(chembls, name="CHEMBL")
raw_df = raw_df.drop(columns=['inhibitor_panel', 'replicate', 'well_concentration'])
full_df = raw_df.merge(cids, left_on='inhibitor', right_index=True).merge(chembls, left_on='inhibitor', right_index=True, how='left')
# The required columns are "dose", "response", "sample", and "drug", with an optional "replicate" column.
full_df.rename(columns={'dbgap_subject_id': 'sample', 'inhibitor': 'drug_name', 'rounddose': 'dose', 'CID': 'drug', 'normalized_viability': 'response', 'repl': 'replicate'}, inplace=True)
full_df.to_csv('viabilities_processed.csv', index=None)
full_df

In [ ]:
# check for duplicates
print(len(full_df))
print(len(full_df[["sample", "drug", "dose", "response", "replicate"]].drop_duplicates()))
full_df[full_df[["sample", "drug", "dose", "response", "replicate"]].duplicated()]

## DrEvalPy: run CurveCurator

In [ ]:
preprocess(input_file='viabilities_processed.csv', output_dir='curvecurator', dataset_name='BeatAML2', cores=6, normalize=True)

In [ ]:
_exec_curvecurator(Path('curvecurator'))

In [ ]:
postprocess('curvecurator', dataset_name='BeatAML2')

## Postprocess for Zenodo

* Quality control
* Set EC50s outside the measured range to NaN
* Set IC50s outside 10*max/10**-1 min measured range to NaN


In [ ]:
viability_df = pd.read_csv('curvecurator/Bruna.csv', converters={'sample': str, 'pubchem_id': str})
full_df = pd.read_csv('viabilities_processed.csv', converters={'drug': str})[['sample', 'drug', 'drug_name', 'CHEMBL', 'mindose', 'maxdose', ]].drop_duplicates()
viability_df = viability_df.merge(full_df, left_on=['cell_line_name', 'pubchem_id'], right_on=['sample', 'drug'])

In [ ]:
# join with original data
original_measures_df = pd.read_csv('beataml_probit_curve_fits_v4_dbgap.txt', sep='\t')
viability_df = viability_df.merge(original_measures_df, left_on=['cell_line_name', 'drug_name'], right_on=['dbgap_subject_id', 'inhibitor'])
viability_df = viability_df.rename(columns={'ic50': 'IC50', 'auc': 'AUC'})

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
# checks out because they measure AUC=area under the dose response curve, we measure AUC=area under the dose viability curve
sns.scatterplot(viability_df, x="AUC_curvecurator", y="AUC", s=1)

In [ ]:
sns.scatterplot(viability_df, x="IC50_curvecurator", y="IC50", s=1)
plt.xlim(1e-4, 1e2)
plt.ylim(1e-4, 1e2)
plt.xscale('log')
plt.yscale('log')

In [ ]:
# create cell line names file
response_data = data.merge(df_map, left_on="inhibitor", right_on="drug_name_beataml", how="left")
response_data["LN_IC50"] = response_data["ic50"]
response_data.drop(columns=["ic50", "drug_name_beataml", "status", "paper_inclusion"], inplace=True)
response_data["cell_line_name"] = response_data["dbgap_subject_id"].astype(str)

cell_line_names = response_data.drop_duplicates("cell_line_name")[["cell_line_name"]]
cell_line_names["Cellosaurus_id"] = pd.NA
cell_line_names["tissue"] = "Blood"
cell_line_names.to_csv("cell_line_names.csv", index=False)

In [ ]:
# create gene expression feature file
gene_exp = pd.read_csv("other/beataml_waves1to4_norm_exp_dbgap.txt", sep="\t", index_col=0)

gene_exp_processed = gene_exp
gene_exp_processed =  gene_exp_processed.reset_index().drop(columns=["stable_id", "description", "biotype"]).set_index("display_label")
gene_exp_processed = gene_exp_processed.T
map_rna_id_to_cl_name = {rna_ind: sub_ind for sub_ind, rna_ind in zip(response_data["cell_line_name"], response_data["dbgap_rnaseq_sample"])}
#remap index
gene_exp_processed.index = [map_rna_id_to_cl_name.get(ind, pd.NA) for ind in gene_exp_processed.index]
gene_exp_processed =  gene_exp_processed.drop(pd.NA)
gene_exp_processed.to_csv("gene_expression.csv", index=True)

In [ ]:
# create drug names file
drug_names = df_map[["drug_name_beataml", "drug_name", "pubchem_id"]]
#if pubchemid is NA, use drug_name
drug_names["pubchem_id"] = drug_names["pubchem_id"].replace("", pd.NA)
drug_names["pubchem_id"] = drug_names["pubchem_id"].replace(" ", pd.NA)

drug_names["pubchem_id"] = drug_names["pubchem_id"].fillna(drug_names["drug_name"])

drug_names.to_csv("drug_names.csv", index=False)

In [ ]:
#create fingerprints
import os
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs

# df has columns: pubchem_id, canonical_smiles
drug_smiles = pd.read_csv("drug_smiles.csv")

df = drug_smiles.copy()

out_dir = "drug_fingerprints"
os.makedirs(out_dir, exist_ok=True)

def morgan_bits(smiles, n_bits, radius=2):
    m = Chem.MolFromSmiles(smiles)
    if m is None:
        return np.full(n_bits, np.nan)  # keep shape even if invalid
    fp = AllChem.GetMorganFingerprintAsBitVect(m, radius, nBits=n_bits)
    arr = np.zeros((n_bits,), dtype=int)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

sizes = [64, 128, 256, 512, 1024, 2048]

for n in sizes:
    print(f"computing {n}-bit fingerprints")
    # compute and stack into a DataFrame (rows = bit positions, cols = pubchem_id)
    fp_matrix = {}
    for pid, smi in zip(df["pubchem_id"], df["canonical_smiles"]):
        fp_matrix[pid] = morgan_bits(smi, n)
    fp_df = pd.DataFrame(fp_matrix)  # index 0..n-1, columns = pubchem_id
    fp_df.to_csv(f"{out_dir}/pubchem_id_to_demorgan_{n}_map.csv", index=False)


In [ ]:
# rename genes in gene expression file  to correct symbols
ge = pd.read_csv("gene_expression.csv", index_col=0)
genes = list(ge.columns)
new_gene_names = {
    "AARS": "AARS1",
    "EPRS": "EPRS1",
    "FAM57A": "TLCD3A",
    "FAM69A": "DIPK1A",
    "H2AFV": "H2AZ2",
    "HIST1H2BK": "H2BC12",
    "HIST2H2BE": "H2BC21",
    "KIAA0100": "BLTP2",
    "KIAA0355": "GARRE1",
    "NARFL": "CIAO3",
    "PAPD7": "TENT4A",
    "SKIV2L": "SKIC2",
    "TSTA3": "GFUS",
    "WDR61": "SKIC8",
    "WRB": "GET1",
}
new_genes = [new_gene_names.get(g, g) for g in genes]
ge.columns = new_genes
landmark_genes  = pd.read_csv("gene_lists/landmark_genes.csv", index_col=0)
set(landmark_genes["Symbol"]) - set(new_genes)
#save again
ge.index.name = "cell_line_name"
ge.to_csv("gene_expression.csv", index=True)

In [ ]:
# correct pubchem ids in BeatAML2.csv
import pandas as pd

d = pd.read_csv("BeatAML2.csv")
d["pubchem_id"] = d["pubchem_id"].astype(str).str.replace(".0", "", regex=False)

d.loc[d["drug_name"] == "MEK1/2 Inhibitor", "pubchem_id"] = "MEK1/2 Inhibitor"
d.loc[d["drug_name"] == "JQ12", "pubchem_id"] = "JQ12"

d.to_csv("BeatAML2.csv", index=False)
